# 190116, 영한번역 translate.py 커스터마이징
본래 코드가 짜쯩나서 쥬피터로 다시짬.
val 데이타로 번역하기
src, tgt 동시에 보여줘서 비교할수 있게 함, txt로도 출력

In [1]:
import argparse
import sys, time
import codecs
from operator import itemgetter

import torch
from data_loader import DataLoader
import data_loader
from simple_nmt.seq2seq import Seq2Seq

In [2]:
# parameter define
config_model      = 'models/enko.pth'
config_src_val    = 'data/corpus.valid.en'
config_tgt_val    = 'data/corpus.valid.ko'

config_gpu_id     = 0  # 0 : gpu 사용, -1 : 미사용
config_batch_size = 128
config_max_length = 255
config_n_best     = 1
config_beam_size  = 5

In [3]:
def read_text_line(data):
    f = open(data)
    tgt_data = f.read()
    tgt_data_line = tgt_data.split('\n')
    # line 단위 list
    return tgt_data_line

def read_text_vocab(data):
    lines = []
    f = open(data)
    src_data = f.read()
    src_data_line = src_data.split('\n')
    for line in src_data_line:
        if line.strip() != '':
            lines += [line.strip().split(' ')]
    return lines

def to_text(indice, vocab):
    # This method converts index to word to show the translation result.
    lines = []
    for i in range(len(indice)):
        line = []
        for j in range(len(indice[i])):
            index = indice[i][j]
            if index == data_loader.EOS:
                # line += ['<EOS>']
                break
            else:
                line += [vocab.itos[index]]
        line = ' '.join(line)
        lines += [line]
    return lines

In [4]:
# Load saved model.
saved_data = torch.load(config_model, map_location='cpu' if config_gpu_id < 0 else 'cuda:%d' % config_gpu_id)
src_vocab = saved_data['src_vocab']
tgt_vocab = saved_data['tgt_vocab']

train_config = saved_data['config']
# Initialize dataloader, but we don't need to read training & test corpus.
# What we need is just load vocabularies from the previously trained model.
loader = DataLoader()
loader.load_vocab(src_vocab, tgt_vocab)
input_size = len(loader.src.vocab)
output_size = len(loader.tgt.vocab)

In [5]:
# Declare sequence-to-sequence model.
model = Seq2Seq(input_size,
                train_config.word_vec_dim,
                train_config.hidden_size,
                output_size,
                n_layers=train_config.n_layers,
                dropout_p=train_config.dropout
                )
model.load_state_dict(saved_data['model'])  # Load weight parameters from the trained model.
model.eval()  # We need to turn-on the evaluation mode, which turns off all drop-outs.

# We don't need to draw a computation graph, because we will have only inferences.
torch.set_grad_enabled(False)

# Put models to device if it is necessary.
if config_gpu_id >= 0:
    print('gpu 사용')
    model.cuda(config_gpu_id)

gpu 사용


In [6]:
start_time = time.time();
# Get sentences from standard input.
lines = read_text_vocab(config_src_val)
lines_src = read_text_line(config_src_val)
lines_tgt = read_text_line(config_tgt_val)

config_batch_size = len(lines_src)

with torch.no_grad():  # Also, declare again to prevent to get gradients.
    while len(lines) > 0:
        # Since packed_sequence must be sorted by decreasing order of length,
        # sorting by length in mini-batch should be restored by original order.
        # Therefore, we need to memorize the original index of the sentence.
        sorted_lines = lines[:config_batch_size]
        lines = lines[config_batch_size:]

        lengths = [len(_) for _ in sorted_lines]  # 문장 길이 list로
        orders = [i for i in range(len(sorted_lines))]

        sorted_tuples = sorted(zip(sorted_lines, lengths, orders), 
                               key=itemgetter(1),
                               reverse=True
                               )
        sorted_lines = [sorted_tuples[i][0] for i in range(len(sorted_tuples))]
        lengths = [sorted_tuples[i][1] for i in range(len(sorted_tuples))]
        orders = [sorted_tuples[i][2] for i in range(len(sorted_tuples))]

        # Converts string to list of index.
        x = loader.src.numericalize(loader.src.pad(sorted_lines),
                                    device='cuda:%d' % config_gpu_id if config_gpu_id >= 0 else 'cpu'
                                    )

        if config_beam_size == 1:
            # Take inference for non-parallel beam-search.
            y_hat, indice = model.search(x)
            output = to_text(indice, loader.tgt.vocab)

            sorted_tuples = sorted(zip(output, orders), key=itemgetter(1))
            output = [sorted_tuples[i][0] for i in range(len(sorted_tuples))]
            
            sys.stdout.write('\n'.join(output) + '\n')

        else:
            # Take mini-batch parallelized beam search.
            batch_indice, _ = model.batch_beam_search(x,
                                                      beam_size=config_beam_size,
                                                      max_length=config_max_length,
                                                      n_best=config_n_best
                                                      )

            # Restore the original orders.
            output = []
            for i in range(len(batch_indice)):
                output += [to_text(batch_indice[i], loader.tgt.vocab)]
            sorted_tuples = sorted(zip(output, orders), key=itemgetter(1))
            output = [sorted_tuples[i][0] for i in range(len(sorted_tuples))]
print('translation complete');print("--- %s seconds ---" %(time.time() - start_time))
print('text로 출력')
f = open("data/val_set.txt", 'w')
for i in range(len(output)):
    f.write(lines_src[i])
    f.write('\n')
    f.write(lines_tgt[i])
    f.write('\n')
    f.write(output[i][0])
    f.write('\n\n')
f.close()

translation complete
--- 2.4324090480804443 seconds ---
text로 출력


In [7]:
for i in range(0,30):
    print('=======================================')
    print('src:',lines_src[i])
    print('tgt:',lines_tgt[i])
    print('nmt:',output[i][0])
    print('=======================================')

src: IF AN OUT OF SEQUENCE COUPON IS PRESENTED FOR REFUND THE TICKET IS NON-REFUNDABLE STATUTORY TAXES GOVT.
tgt: 같은 RBD내에서 적용된 편도 운임이 존재하지 않는다면 그보다 높은 RBD가 적용되고 취소 수수료가 부과됩니다.
nmt: 이것은 사용된 항공권의 하나라도 있는 경우 환불 가능합니다.
src: WAIVED FOR DEATH OF PASSENGER OR FAMILY MEMBER.
tgt: 탑승객 혹은 직계 가족 구성원의 사망 시 면제됩니다.
nmt: 탑승객 혹은 직계 가족 구성원의 사망시 수수료가 면제됩니다.
src: CHANGE FEE CAD100  -CHANGE FEE TO BE COLLECTED PER TRANSACTION FOR REBOOKING REROUTING UPGRADING OR ANY CHANGE REQUIRING TICKET TO BE REISSUED.
tgt: 각 거래당 재예약,여정변경, 업그레이드 혹은 모든 항공권의 재발행을 요구하는 변경은 변경 수수료 CAD100가 부과됩니다.
nmt: 변경 수수료 USD 1 -재예약 여정 변경 업그레이드 또는 항공권 재발행을 요하는 모든 다른 변경은 각 거래에 대한 변경 수수료가 적용됩니다.
src: EXTENSION OF TICKET VALIDITY IS NOT PERMITTED.
tgt: 항공권 연장은 불가능합니다.
nmt: 여정이 연장은 허용되지 않습니다.
src: FOR PARTIALLY UTILIZED TICKETS HISTORICAL FARES MUST BE USED TO REPRISE/REISSUE THE TICKET.
tgt: 부분 사용한 항공권의 경우 이전 운임을 반드시 사용하여 재계산/재발행 됩니다.
nmt: 부분 사용 항공권 - 모든 기존 환불 불가능한 요금은 계속 재책정됩니다.
src: FARE CAN BE UPGRADED TO ANY EQUAL OR HIGHER FARE.
tgt: 